# Modeling and optimization of a weekly workforce with Python and Pyomo
Step-by-step modeling and optimization of a workforce design and assignation problem using Python and Pyomo.
Code of the post https://towardsdatascience.com/modeling-and-optimization-of-a-weekly-workforce-with-python-and-pyomo-29484ba065bb.

## Problem description
A new food store has been opened at the University Campus which will be open 24 hours a day, 7 days a week. Each day, there are three eight-hour shifts. Morning shift is from 6:00 to 14:00, evening shift is from 14:00 to 22:00 and night shift is from 22:00 to 6:00 of the next day.
During the night there is only one worker while during the day there are two, except on Sunday that there is only one for each shift. Each worker will not exceed a maximum of 40 hours per week and have to rest for 12 hours between two shifts.
As for the weekly rest days, an employee who rests one Sunday will also prefer to do the same that Saturday.
In principle, there are available ten employees, which is clearly over-sized. The less the workers are needed, the more the resources for other stores.

In [31]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [32]:
# Define model parameters
# Define days (1 week)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Enter shifts of each day
shifts = ['morning', 'evening', 'night']  # 3 shifts of 8 hours
days_shifts = {day: shifts for day in days}  # dict with day as key and list of its shifts as value

# Enter workers ids (name, number, ...)
workers = ['W' + str(i) for i in range(1, 11)]  # 10 workers available, more than needed

In [33]:
workers

['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']

In [34]:
days_shifts

{'Mon': ['morning', 'evening', 'night'],
 'Tue': ['morning', 'evening', 'night'],
 'Wed': ['morning', 'evening', 'night'],
 'Thu': ['morning', 'evening', 'night'],
 'Fri': ['morning', 'evening', 'night'],
 'Sat': ['morning', 'evening', 'night'],
 'Sun': ['morning', 'evening', 'night']}

In [35]:
days

['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [36]:
shifts

['morning', 'evening', 'night']

In [37]:
# Initialize model and variables
# Initialize model
model = ConcreteModel()

In [38]:
# binary variables representing if a worker is scheduled somewhere
model.works = Var(((worker, day, shift) for worker in workers for day in days for shift in days_shifts[day]),
                  within=Binary, initialize=0)

works: binary variable indexed by workers, days and shifts. 1 if the worker has to work that day on that shift.

In [39]:
# binary variables representing if a worker is necessary
model.needed = Var(workers, within=Binary, initialize=0)

needed: binary variable indexed by workers. 1 if the worker is necessary to include in the workforce.

In [40]:
# binary variables representing if a worker worked on sunday but not on saturday (avoid if possible)
model.no_pref = Var(workers, within=Binary, initialize=0)

no_pref: binary variable indexed by workers. 1 if it does not work on Sunday but it does on Saturday.

In [41]:
model

In [42]:
model.works

In [43]:
model.needed

In [44]:
model.no_pref

In [45]:
# Define the objective function
# Define an objective function with model as input, to pass later
def obj_rule(m):
    c = len(workers)
    return sum(m.no_pref[worker] for worker in workers) + sum(c * m.needed[worker] for worker in workers)
# we multiply the second term by a constant to make sure that it is the primary objective
# since sum(m.no_prefer) is at most len(workers), len(workers) + 1 is a valid constant.


# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_rule, sense=minimize)

The objective is to find a schedule that minimizes the number of workers and once this is achieved, also the number of workers that work on Sundays but not on Saturdays. We multiplied the part of the number of workers by a constant big enough so that minimizing the weekend preference is considered only after deciding the optimal number of workers.

We also need to add the constraints, creating first a container of constraints calling ConstraintList() and then adding whatever constraints we want to the container with the function add. Code for the constraints is the following, with the explanation of each one as inline comments:

In [46]:
# Add model constraints
model.constraints = ConstraintList()  # Create a set of constraints

# Constraint: all shifts are assigned
for day in days:
    for shift in days_shifts[day]:
        if day in days[:-1] and shift in ['morning', 'evening']:
            # weekdays' and Saturdays' day shifts have exactly two workers
            model.constraints.add(  # to add a constraint to model.constraints set
                2 == sum(model.works[worker, day, shift] for worker in workers)
            )
        else:
            # Sundays' and nights' shifts have exactly one worker
            model.constraints.add(
                1 == sum(model.works[worker, day, shift] for worker in workers)
            )

# Constraint: no more than 40 hours worked
for worker in workers:
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

# Constraint: rest between two shifts is of 12 hours (i.e., at least two shifts)
for worker in workers:
    for j in range(len(days)):
        # if working in morning, cannot work again that day
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in days_shifts[days[j]])
        )
        # if working in evening, until next evening (note that after sunday comes next monday)
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in ['evening', 'night']) +
            model.works[worker, days[(j + 1) % 7], 'morning']
        )
        # if working in night, until next night
        model.constraints.add(
            1 >= model.works[worker, days[j], 'night'] +
            sum(model.works[worker, days[(j + 1) % 7], shift] for shift in ['morning', 'evening'])
        )

# Constraint (def of model.needed)
for worker in workers:
    model.constraints.add(
        10000 * model.needed[worker] >= sum(model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )  # if any model.works[worker, ·, ·] non-zero, model.needed[worker] must be one; else is zero to reduce the obj function
    # 10000 is to remark, but 5 was enough since max of 40 hours yields max of 5 shifts, the maximum possible sum

# Constraint (def of model.no_pref)
for worker in workers:
    model.constraints.add(
        model.no_pref[worker] >= sum(model.works[worker, 'Sat', shift] for shift in days_shifts['Sat'])
        - sum(model.works[worker, 'Sun', shift] for shift in days_shifts['Sun'])
    )  # if not working on sunday but working saturday model.needed must be 1; else will be zero to reduce the obj function

In [47]:
model.pprint()

4 Set Declarations
    constraints_index : Dim=0, Dimen=1, Size=261, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 

Now, the model created can be solved. A model can be visualized with model.pprint(). This problem is composed of binary variables, so a Mixed Integer Programming solver suits our requirements and CBC from COIN-OR will be selected, https://projects.coin-or.org/Cbc.

# Find the solutions
Local solver:
Need to have installed cbc solver and added the executable to the path. One easy way to do this is to download it from https://ampl.com/products/solvers/open-source/#cbc for your particular OS, put the files into a folder and then add that folder to your environment variables.

In [48]:
opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
results = solver_manager.solve(model, opt=opt)

In [49]:
# Extract the solution
def get_workers_needed(needed):
    """Extract to a list the needed workers for the optimal solution."""
    workers_needed = []
    for worker in workers:
        if needed[worker].value == 1:
            workers_needed.append(worker)
    return workers_needed


def get_work_table(works):
    """Build a timetable of the week as a dictionary from the model's optimal solution."""
    week_table = {day: {shift: [] for shift in days_shifts[day]} for day in days}
    for worker in workers:
        for day in days:
            for shift in days_shifts[day]:
                    if works[worker, day, shift].value == 1:
                        week_table[day][shift].append(worker)
    return week_table


def get_no_preference(no_pref):
    """Extract to a list the workers not satisfied with their weekend preference."""
    return [worker for worker in workers if no_pref[worker].value == 1]


workers_needed = get_workers_needed(model.needed)  # dict with the optimal timetable
week_table = get_work_table(model.works)  # list with the required workers
workers_no_pref = get_no_preference(model.no_pref)  # list with the non-satisfied workers (work on Sat but not on Sun)

In [50]:
# Visualize solution as json
import json
print('Workers needed:')
[print('  ' + worker) for worker in workers_needed];

Workers needed:
  W1
  W2
  W3
  W4
  W5
  W6
  W10


In [51]:
print('Work schedule:')
print(json.dumps(week_table, indent=2))

Work schedule:
{
  "Mon": {
    "morning": [
      "W5",
      "W10"
    ],
    "evening": [
      "W2",
      "W4"
    ],
    "night": [
      "W3"
    ]
  },
  "Tue": {
    "morning": [
      "W1",
      "W6"
    ],
    "evening": [
      "W4",
      "W10"
    ],
    "night": [
      "W3"
    ]
  },
  "Wed": {
    "morning": [
      "W1",
      "W6"
    ],
    "evening": [
      "W2",
      "W10"
    ],
    "night": [
      "W5"
    ]
  },
  "Thu": {
    "morning": [
      "W1",
      "W4"
    ],
    "evening": [
      "W2",
      "W10"
    ],
    "night": [
      "W6"
    ]
  },
  "Fri": {
    "morning": [
      "W1",
      "W3"
    ],
    "evening": [
      "W2",
      "W4"
    ],
    "night": [
      "W10"
    ]
  },
  "Sat": {
    "morning": [
      "W5",
      "W6"
    ],
    "evening": [
      "W1",
      "W3"
    ],
    "night": [
      "W2"
    ]
  },
  "Sun": {
    "morning": [
      "W5"
    ],
    "evening": [
      "W6"
    ],
    "night": [
      "W3"
    ]
  }
}


In [52]:
print('Workers not satisfied by weekend condition:')
[print('  ' + worker) for worker in workers_no_pref];

Workers not satisfied by weekend condition:
  W1
  W2


In [53]:
print('Objective value:', model.obj())

Objective value: 72.0
